In [ ]:
import pandas as pd

In [ ]:
 import sqlite3

In [ ]:
conn = sqlite3.connect(':memory:')

# 1 задача

- sql

In [ ]:
query = """-- Удаление таблицы, если она существует
DROP TABLE IF EXISTS t1;
-- Создание таблицы с полями key, id, phone, mail
CREATE TABLE IF NOT EXISTS t1 ( `key` INT, `id` INT, `phone` VARCHAR(20), `mail` VARCHAR(50) );
-- Вставка тестовых данных
INSERT INTO t1 (`key`, `id`, `phone`, `mail`) VALUES (1, 12345, '87778885566', 'one@mail.ru'), (2, 54321, '87778885566', 'two@mail.ru'), (3, 67890, '87770008899', 'three@mail.ru'), (4, 66678, '87778885566', 'four@mail.ru'), (5, 34567, '84547895566', 'four@mail.ru'), (6, 34567, '89087545678', 'five@mail.ru');
-- Запрос для поиска всех связанных данных по указанному phone
SELECT * FROM t1 WHERE phone = '87778885566' OR id IN ( SELECT id FROM t1 WHERE phone = '87778885566' )"""

# Выполняем SQL-запрос
cursor = conn.cursor()
cursor.executescript(query)
# Используем executescript для выполнения нескольких SQL-запросов
conn.commit()
# Выполняем запрос на выборку
df = pd.read_sql_query("SELECT * FROM t1 WHERE phone = '87778885566' OR id IN (SELECT id FROM t1 WHERE phone = '87778885566')", conn)
# Закрываем соединение conn.close()
# Вывод результата
display(df)

,key,id,phone,mail
0,1,12345,87778885566,one@mail.ru
1,2,54321,87778885566,two@mail.ru
2,4,66678,87778885566,four@mail.ru


- python

In [ ]:
# Создание DataFrame с тестовыми данными
data = { 'key': [1, 2, 3, 4, 5, 6], 'id': [12345, 54321, 67890, 66678, 34567, 34567], 'phone': ['87778885566', '87778885566', '87770008899', '87778885566', '84547895566', '89087545678'], 'mail': ['one@mail.ru', 'two@mail.ru', 'three@mail.ru', 'four@mail.ru', 'four@mail.ru', 'five@mail.ru'] }
df = pd.DataFrame(data)
# Условие поиска по phone
phone_condition = '87778885566'
# Поиск всех строк, где phone совпадает с заданным или id связан с таким phone
matching_ids = df[df['phone'] == phone_condition]['id'].unique()
# Получаем все id, которые имеют указанный phone
result = df[(df['phone'] == phone_condition) | (df['id'].isin(matching_ids))]
# Фильтруем по phone или связанным id
# Вывод результата
display(result)

,key,id,phone,mail
0,1,12345,87778885566,one@mail.ru
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru


# 2 задача

- sqllite

In [ ]:
# Создаем таблицы и вставляем данные
query = """ -- Создание таблицы клиентов
CREATE TABLE IF NOT EXISTS CLIENTS_TABLE ( CLIENT_ID int, CLIENT_NAME NVARCHAR(20), BIRTHDAY date, GENDER NVARCHAR(20) );
-- Создание таблицы займов
CREATE TABLE IF NOT EXISTS LOANS_TABLE ( LOAN_ID int, CLIENT_ID int, LOAN_DATE date, LOAN_AMOUNT float );
-- Вставка данных в таблицу клиентов
INSERT INTO CLIENTS_TABLE (CLIENT_ID, CLIENT_NAME, BIRTHDAY, GENDER) VALUES (1, 'bob', '2020-01-15', 'male'), (2, 'rocky', '2020-02-15', 'female'), (3, 'like', '2020-02-15', 'female'), (4, 'ricky', '2020-02-15', 'male');
-- Вставка данных в таблицу займов
INSERT INTO LOANS_TABLE (LOAN_ID, CLIENT_ID, LOAN_DATE, LOAN_AMOUNT) VALUES (1, 1, '2020-01-15', 10000), (2, 2, '2020-02-15', 20000), (3, 3, '2020-03-15', 30000), (4, 4, '2020-04-15', 40000), (5, 1, '2020-01-16', 15000), (6, 2, '2020-03-15', 35000), (7, 3, '2020-03-15', 5000), (8, 1, '2020-01-15', 1500), (9, 2, '2020-01-15', 500), (10, 1, '2020-01-15', 1500); """

# Выполняем запросы для создания таблиц и вставки данных
conn.executescript(query)
conn.commit()
# Запрос для выполнения выборки с "нумерацией" займов и подсчетом количества
query = """ WITH RankedLoans AS ( SELECT LOAN_ID,
                                         CLIENT_ID,
                                         LOAN_DATE,
                                         LOAN_AMOUNT,
                                         ( SELECT COUNT(*) + 1
                                         FROM LOANS_TABLE AS sub WHERE sub.CLIENT_ID = LOANS_TABLE.CLIENT_ID AND sub.LOAN_DATE < LOANS_TABLE.LOAN_DATE ) AS loan_rank
                                  FROM LOANS_TABLE )
            SELECT c.GENDER,
                   CASE
                     WHEN rl.loan_rank = 1
                     THEN 'Первый договор'
                     WHEN rl.loan_rank = 2
                     THEN 'Второй договор'
                     WHEN rl.loan_rank = 3
                     THEN 'Третий договор'
                     WHEN rl.loan_rank = 4
                     THEN 'Четвертый договор' END AS договор,
                    COUNT(*) AS договор_count
            FROM RankedLoans AS rl JOIN CLIENTS_TABLE AS c ON rl.CLIENT_ID = c.CLIENT_ID WHERE rl.loan_rank <= 4
            GROUP BY c.GENDER, CASE WHEN rl.loan_rank = 1 THEN 'Первый договор' WHEN rl.loan_rank = 2 THEN 'Второй договор' WHEN rl.loan_rank = 3 THEN 'Третий договор' WHEN rl.loan_rank = 4 THEN 'Четвертый договор' END ORDER BY c.GENDER, договор; """
# Выполнение запроса и получение результата в DataFrame
df = pd.read_sql_query(query, conn)
# Вывод результата
display(df)
 # Закрываем соединение
conn.close()

,GENDER,договор,договор_count
0,female,Второй договор,1
1,female,Первый договор,3
2,female,Третий договор,1
3,male,Первый договор,4
4,male,Четвертый договор,1


- posgres

In [ ]:
# Пронумеровываем займы для каждого клиента по дате (по возрастанию)
query = """WITH RankedLoans AS (SELECT LOAN_ID,
                                      CLIENT_ID,
                                      LOAN_DATE,
                                      LOAN_AMOUNT,
                                      ROW_NUMBER() OVER (PARTITION BY CLIENT_ID ORDER BY LOAN_DATE) AS loan_rank
                                FROM LOANS_TABLE )
-- Извлекаем первый, второй, третий и четвертый займы для каждого клиента
SELECT c.GENDER,
       CASE
         WHEN rl.loan_rank = 1
         THEN 'Первый договор'
         WHEN rl.loan_rank = 2
         THEN 'Второй договор'
         WHEN rl.loan_rank = 3
         THEN 'Третий договор'
         WHEN rl.loan_rank = 4
         THEN 'Четвертый договор' END AS договор,
      COUNT(*) AS договор_count
FROM RankedLoans AS rl JOIN CLIENTS_TABLE AS c ON rl.CLIENT_ID = c.CLIENT_ID
WHERE rl.loan_rank <= 4
-- Отбираем только первый, второй, третий и четвертый договоры
GROUP BY c.GENDER,
         CASE WHEN rl.loan_rank = 1 THEN 'Первый договор' WHEN rl.loan_rank = 2 THEN 'Второй договор' WHEN rl.loan_rank = 3 THEN 'Третий договор' WHEN rl.loan_rank = 4 THEN 'Четвертый договор' END ORDER BY c.GENDER, договор;"""